# Simulate Annealing for Single Instances

In [ ]:
using QAOA, Distributions, Interpolations, LinearAlgebra
using HDF5, Printf
using PythonPlot
PythonPlot.matplotlib.style.use("./paper.mplstyle")
PythonPlot.rc("axes", prop_cycle=PythonPlot.matplotlib.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))

# PATH = "../";
PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.8f", f)

In [ ]:
# N = 9
N = 11

patterns_dict = Dict(
    9  => r"random_SK_instance_N_9_seed_(\d+)\.h5",
    11 => r"random_SK_instance_N_11_seed_(\d+)\.h5",
    13 => r"random_SK_instance_N_13_seed_(\d+)\.h5",
    15 => r"random_SK_instance_N_15_seed_(\d+)\.h5",
    17 => r"random_SK_instance_N_17_seed_(\d+)\.h5",
    19 => r"random_SK_instance_N_19_seed_(\d+)\.h5"
);

subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

## Exact Fidelities

In [ ]:
# # N = 9
# seed = 22824
# large_seed = 100210

# N = 11
seed = 2809
large_seed = 100061

J_mat = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "J")
mf_problem = Problem(0, J_mat);

λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_eigvals");

In [ ]:
# # look what's inside
# h5open(PATH * @sprintf("data/SK_model/N_%i/%s/", N, "large_gaps") * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, large_seed))

In [ ]:
T_final = 32768
tol = 1e-8;

In [ ]:
exact_times = range(0, 1, 33);

In [ ]:
gap = λ[2, :] .- λ[1, :];
mingap = minimum(gap) 
mingap |> println
gap_idx = findfirst(x -> x == mingap, gap) 
gaploc = exact_times[gap_idx] 
gaploc |> println

In [ ]:
all_eigvecs = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_lowest_eigvecs");
# all_eigvecs_II = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, 23583), "exact_ARPACK_LM_lowest_eigvecs")

λ_large = h5read(PATH * @sprintf("data/SK_model/N_%i/%s/", N, "large_gaps") * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, large_seed), "exact_ARPACK_LM_eigvals");
all_eigvecs_large = h5read(PATH * @sprintf("data/SK_model/N_%i/%s/", N, "large_gaps") * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, large_seed), "exact_ARPACK_LM_lowest_eigvecs");

In [ ]:
gs = [all_eigvecs[k, :, 1] for k in 1:length(exact_times)]
ex = [all_eigvecs[k, :, 4] for k in 1:length(exact_times)]
# gs_II = [all_eigvecs_II[k, :, 1] for k in 1:length(exact_times)]
gs_large = [all_eigvecs_large[k, :, 1] for k in 1:length(exact_times)]

gs_fidelity = [gs[end]' * gs[k] for k in 1:length(exact_times)];
ex_fidelity = [gs[end]' * ex[k] for k in 1:length(exact_times)];
# gs_fidelity_II = [gs_II[end]' * gs_II[k] for k in 1:length(exact_times)]
gs_fidelity_large = [gs_large[end]' * gs_large[k] for k in 1:length(exact_times)];

In [ ]:
figure(figsize=(4., 2.))
ax = subplot(111)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
plot(exact_times, (λ_large[2, :] .- λ_large[1, :]), "--C0", lw=1, alpha=0.75, label="large gap")
plot(exact_times, (λ[2, :] .- λ[1, :]), "--C1", lw=1.5, alpha=0.75, label="small gap")
# plot(exact_times, abs.(gs_fidelity_II).^2, label="small gap")
plot(exact_times, abs.(gs_fidelity_large).^2, "-sC0", ms=4)
plot(exact_times, abs.(gs_fidelity).^2, "-oC1", lw=1.5, ms=4)
xlim(0., 1.)
ylim(0, 1)
xlabel("\$s\$")
ylabel("\$|\\langle 0\\hspace{0.25mm}|\\hspace{0.25mm}\\psi_0\\rangle|^2\$")

legend(frameon=false, handlelength=1.2, handletextpad=0.4, borderaxespad=0.1, fontsize=12, loc="center left")

tight_layout(pad=0.1)
savefig(PLOT_PATH * @sprintf("fidelity_N_%i_seeds_%i_%i.pdf", N, seed, large_seed), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

## Annealing

In [ ]:
# N = 9
seed = 23583
seed = 131412

# N = 11
seed = 2809

J_mat = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "J")
mf_problem = Problem(0, J_mat);

λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_eigvals");

In [ ]:
sol_dict = Dict();

In [ ]:
# seed = 131412

# T_anneal = 2.0^8
# p = 2^12

# T_anneal = 2.0^9
# p = 2^13

# T_anneal = 2.0^10
# p = 2^14

# T_anneal = 2.0^11
# p = 2^15
;

In [ ]:
# seed = 23583
# seed = 2809

T_anneal = 2.0^13
p = 2^16

T_anneal = 2.0^14
p = 2^17

T_anneal = 2.0^15
p = 2^18

T_anneal = 2.0^16
p = 2^19
;

In [ ]:
sol_dict[T_anneal] = []

In [ ]:
linear_schedule(t) = t / T_anneal

# second-order schedule
τ = T_anneal / p
γ = τ .* ((1:p) .- 1/2) ./ p |> collect
β = τ .* (1 .- (1:p) ./ p) |> collect
β[p] = τ / (4 * p);

In [ ]:
# annealing_problem = Problem(p, zeros(N), J_mat)
annealing_problem = Problem(p, J_mat)

In [ ]:
H_Z = -hamiltonian(0, 1, annealing_problem.local_fields, annealing_problem.couplings);

In [ ]:
# probabs = anneal(annealing_problem, linear_schedule, T_anneal);

In [ ]:
using Yao
beta_and_gamma = vcat(β, γ)
circ = QAOA.circuit(annealing_problem)
circ = QAOA.dispatch_parameters!(circ, annealing_problem, beta_and_gamma)
probabs = Yao.uniform_state(Yao.nqubits(circ)) |> circ |> Yao.probs;

In [ ]:
figure(figsize=(3, 2))
plot(probabs, label=@sprintf("\$T = %0.f\$", T_anneal))
xlim(0, 2^annealing_problem.num_qubits)
ylim(0, 1)
legend(frameon=false)
tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();

In [ ]:
max_prob = maximum(probabs)
max_prob |> println
sol_idxs = findall(x -> x == max_prob, probabs)

In [ ]:
max_prob_2 = maximum(filter(x -> x != max_prob, probabs))
max_prob_2 |> println
sol_idxs_2 = findall(x -> x == max_prob_2, probabs)

In [ ]:
max_prob_3 = maximum(filter(x -> x != max_prob_2, filter(x -> x != max_prob, probabs)))
max_prob_3 |> println
sol_idxs_3 = findall(x -> x == max_prob_3, probabs)

In [ ]:
bitstrings = digits.(0:2^annealing_problem.num_qubits-1, base=2, pad=annealing_problem.num_qubits)
vec_dict = Dict(1 => [1, 0], -1 => [0, 1]);

In [ ]:
1 .- 2bitstrings[sol_idxs[1]] |> println
1 .- 2bitstrings[sol_idxs_2[1]] |> println
1 .- 2bitstrings[sol_idxs_3[1]] |> println

In [ ]:
(1 .- 2bitstrings[sol_idxs[1]]) .* (1 .- 2bitstrings[sol_idxs_2[1]]) |> println

In [ ]:
sol_vec_1 = reduce(kron, [vec_dict[1 - 2z] for z in bitstrings[sol_idxs[1]]]);
sol_vec_2 = reduce(kron, [vec_dict[1 - 2z] for z in bitstrings[sol_idxs_2[1]]]);
sol_vec_3 = reduce(kron, [vec_dict[1 - 2z] for z in bitstrings[sol_idxs_3[1]]]);

In [ ]:
sol_vec_1' * H_Z * sol_vec_1 |> println
sol_vec_2' * H_Z * sol_vec_2 |> println
sol_vec_3' * H_Z * sol_vec_3 |> println

In [ ]:
push!(sol_dict[T_anneal], max_prob)

In [ ]:
push!(sol_dict[T_anneal], max_prob_2)

In [ ]:
max_prob + max_prob_2

### Plotting

In [ ]:
sol_dict = sol_dict |> collect |> sort

In [ ]:
Ts = [T for (T, p) in sol_dict]
P_0 = [p[1] for (T, p) in sol_dict]
P_1 = [p[2] for (T, p) in sol_dict]

figure(figsize=(3, 2))
ax = subplot(111)
# ax.plot(Ts, P_0 .+ P_1, "-k")#, label="\$|0\\rangle\$")
ax.plot(Ts, P_1, "-sC1", label="\$|1\\rangle\$")
ax.plot(Ts, P_0, "-oC0", label="\$|0\\rangle\$")
ax.set_xscale("log", base=2)
ax.set_xticks(Ts)
# ax.set_xlim(Ts[1], Ts[end])
# ax.set_ylim(-0.1, 1.1)
ax.set_ylim(0., 1.)
xlabel("\$T_f\$")
ylabel("Probability")
legend()
tight_layout(pad=0.1)
savefig(PLOT_PATH * @sprintf("annealing_N_%i_seed_%i.pdf", N, seed), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

#### Instance 23583

Annealing results: $P_0 = 0.030$, $P_1 = 0.811$ with second-order annealing schedule at $T_f = 2^{13}$ and $p = 2^{16}$. 

Annealing results: $P_0 = 0.058$, $P_1 = 0.891$ with second-order annealing schedule at $T_f = 2^{14}$ and $p = 2^{17}$. 

Annealing results: $P_0 = 0.113$, $P_1 = 0.884$ with second-order annealing schedule at $T_f = 2^{15}$ and $p = 2^{18}$. 

Annealing results: $P_0 = 0.213$, $P_1 = 0.787$ with second-order annealing schedule at $T_f = 2^{16}$ and $p = 2^{19}$. 


Strings `[-1, 1, -1, 1, -1, -1, 1, -1]` and `[1, -1, -1, -1, 1, 1, -1, 1]` with Hamming distance is 7.

In [ ]:
Ts = [2^(13), 2^(14), 2^(15), 2^(16)]
P_0 = [0.030, 0.058, 0.113, 0.213]
P_1 = [0.811, 0.891, 0.884, 0.787]

figure(figsize=(3, 2))
ax = subplot(111)
# ax.plot(Ts, P_0 .+ P_1, "-k")#, label="\$|0\\rangle\$")
ax.plot(Ts, P_1, "-sC1", label="\$|1\\rangle\$")
ax.plot(Ts, P_0, "-oC0", label="\$|0\\rangle\$")
ax.set_xscale("log", base=2)
ax.set_xticks(Ts)
# ax.set_xlim(Ts[1], Ts[end])
# ax.set_ylim(-0.1, 1.1)
ax.set_ylim(0., 1.)
xlabel("\$T_f\$")
ylabel("Probability")
legend()
tight_layout(pad=0.1)
# savefig(PLOT_PATH * @sprintf("annealing_N_%i_seed_%i.pdf", N, seed), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();